In [ ]:
#ライブラリー読み込み
import numpy as np 
import pandas as pd 

#import sklearn.tree as  DecisionTreeRegressor これでは使えない
#機械学習のモデル作成にsklearnのDecisionTreeRegressorを使用する
from  sklearn.tree import DecisionTreeRegressor




In [ ]:


#訓練データを読み込む
tf_train = pd.read_csv("train.csv")

#テスト用のデータを読み込む
tf_test = pd.read_csv("test.csv")

#正解データを読み込む
tf_gender_submission = pd.read_csv("gender_submission.csv")
#tf_train.head()
tf_train.head()


In [ ]:
#欠損値がある行をすべて抜く
#tf_train = tf_train.dropna(subset=['Age',"Fare"])
#tf_test = tf_test.dropna(subset=['Age',"Fare"])

#欠損値の穴埋めをする(testのときにデータと正解値の行数が一致しなくなるため)
#しかし、テストにつかうデータに対して、欠損値に穴埋めしてよいのかという疑問は残る
tf_train = tf_train.fillna(method="ffill")
tf_test = tf_test.fillna(method="ffill")


In [ ]:

#get csv
#Fareは、testデータに欠損があるため
#drop_column =["Name","PassengerId","SibSp","Cabin","Ticket","Embarked"] #0.20055821371610846
#drop_column =["Name","PassengerId","SibSp","Cabin","Ticket","Embarked","Pclass"] #0.21491228070175442
drop_column =["Name","PassengerId","SibSp","Cabin","Ticket","Embarked","Pclass","Age","Parch"] #0.19311944506819512(試した結果)
#drop_column =["Name","PassengerId","SibSp","Cabin","Ticket","Embarked","Pclass","Age","Parch","Fare"] #0.21401855934746247

#不要と思われるカラムを削除する
tf_train.drop(drop_column ,axis=1, inplace=True)
tf_test.drop(drop_column ,axis=1, inplace=True)





In [ ]:
genders = {'male': 0, 'female': 1} # 辞書を作成
# Sexをgendersを用いて数値に変換
tf_train['Sex'] = tf_train['Sex'].map(genders)
tf_test['Sex'] = tf_test['Sex'].map(genders)





In [ ]:
X_train = tf_train.iloc[:, 1:]
Y_train = tf_train['Survived']

X_train.head()


In [ ]:
tree_model = DecisionTreeRegressor() #max depthによって値が変わる
fit = tree_model.fit(X_train, Y_train)

In [ ]:
#様子を見るためtreeの図を出力する
from sklearn import tree
import graphviz 

#tree.plot_tree(fit)

dot_data = tree.export_graphviz(fit, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("taitanic") 

In [ ]:
#学習おわり
predicted_survived = tree_model.predict(tf_test)
print(tf_test.isnull().any())



In [ ]:
#ここからデータの検証に入る
from sklearn.metrics import mean_absolute_error

#０に近づくほど精度が高いモデルと言える
y = tf_gender_submission.Survived
mean_absolute_error(y, predicted_survived)

#最終的に「0.19311944506819512」という数字が出た(0に行くほど精度が高い。逆に１に近づくほど精度が悪いとも言う)